In [ ]:
import mechanism
import pandas as pd
import networkx as nx
from mbi.junction_tree import JunctionTree
from mbi import Domain
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import spectral_clustering
import itertools

In [ ]:
# find the best queries to ask by non-privately analyzing the provisional dataset (assumed to be public)

data = 'competitor_pack/data/colorado.csv'
specs = 'competitor_pack/data/colorado-specs.json'

M = mechanism.Mechanism(data, specs)

data = M.load_data()
domain = M.domain

In [ ]:
# calculate mutual info between every pair of attributes

N = data.df.shape[0]
df = pd.DataFrame(columns=['col1','col2', 'score'])
idx = 0

for a,b in itertools.combinations(data.domain.attrs, 2):
    A = data.project(a).datavector() / N
    B = data.project(b).datavector() / N
    AB = data.project([a,b]).datavector() / N
    nz = AB > 0

    score = np.dot(AB[nz], np.log(AB / np.outer(A,B).flatten())[nz])
    print(a,b,score)
    df.loc[idx] = [a,b,score]
    df.loc[idx+1] = [b,a,score]
    idx += 2

df = df.sort_values('score')


In [ ]:
dom2 = Domain.fromdict(dict(data.df.nunique()))
nodes = domain.attrs
weights = df[df.col1.isin(nodes) & df.col2.isin(nodes)]
weights.loc[(df.col1 == 'SEX') & (df.col2 == 'CITY'), 'score'] += 100
weights.loc[(df.col1 == 'SEX') & (df.col2 == 'INCWAGE'), 'score'] += 100
weights.loc[(df.col1 == 'CITY') & (df.col2 == 'INCWAGE'), 'score'] += 100
weights.loc[(df.col1 == 'CITY') & (df.col2 == 'SEX'), 'score'] += 100
weights.loc[(df.col1 == 'INCWAGE') & (df.col2 == 'SEX'), 'score'] += 100
weights.loc[(df.col1 == 'INCWAGE') & (df.col2 == 'CITY'), 'score'] += 100

G = nx.Graph()
G.add_nodes_from(nodes)

for e1, e2, w in zip(weights['col1'], weights['col2'], weights['score']):
    G.add_edge(e1, e2, weight=w)
    
mst = nx.maximum_spanning_tree(G)
print(list(mst.edges))

In [ ]:
res = pd.DataFrame(columns=['A','B','C','size','score'])
i = 0
N = data.df.shape[0]

for A in nodes:
    for B, C in itertools.combinations(mst.neighbors(A), 2):
        AB = data.df.groupby([A,B]).size().unstack().fillna(0).values / N
        AC = data.df.groupby([A,C]).size().unstack().fillna(0).values / N
        a,b = AB.shape
        a,c = AC.shape
        AB, AC = AB.reshape(a,b,1), AC.reshape(a,1,c)
        
        ABC2 = AB * AC / AB.sum(axis=1, keepdims=True)
        
        ABC = data.df.groupby([A,B,C]).size().unstack().fillna(0).unstack().fillna(0).stack().stack().values.reshape(a,b,c) / N

        score = np.abs(ABC - ABC2).sum()
        #AB = data.project([A,B]).datavector()
        #AC = data.project([A,C]).datavector()
        print(A,B,C)
        res.loc[i] = [A,B,C,a*b*c,score]
        i = i + 1
        
#res[res.score >= 0.05].sort_values('size',ascending=True)

In [ ]:
print([tuple(row) for row in res.loc[res.score >= 0.10,['A','B','C']].values])

In [ ]:
ans = []
for col in nodes:
    lookup = res.loc[res.A == col]
    G = nx.Graph()
    G.add_nodes_from(mst.neighbors(col))

    for e1, e2, w in zip(lookup['B'], lookup['C'], lookup['score']):
        if w >= 0.1:
            G.add_edge(e1, e2, weight=w)

    tree = nx.maximum_spanning_tree(G)
    ans += list(tree.edges)
    ans += [(col,) + e for e in tree.edges]
print(ans)